In [186]:
# importing packages
import os
import pandas as pd
from tqdm import tqdm

In [187]:
stats_path = os.path.abspath('./data/stats')
files = os.listdir(stats_path)

for i in range(len(files)):
    files[i] = os.path.join(stats_path, files[i])

In [188]:
# Making mask for the pre years and post years
pre_mask = pd.DataFrame(['2009-10','2010-11','2011-12','2012-13','2013-14'], columns=['Season']).set_index('Season')
post_mask = pd.DataFrame(['2015-16','2016-17','2017-18','2018-19','2021-22'], columns=['Season']).set_index('Season')

In [189]:
dq = {'pre': [],
     'post': []}

In [190]:
def data_split(df, mask, split_name, file_name):
    split_df = df.join(mask, how='inner').reset_index().drop(['Season', 'Age', 'Tm', 'Lg'], axis=1)
    
    base_path = os.path.abspath('./data/cleaned-stats')
    directory_path = os.path.join(base_path, split_name)
    file_path = os.path.join(directory_path, file_name)
    
    if len(split_df.index) > 0:
        split_df.to_csv(file_path, index=False)
    else:
        dq.get(split_name).append(file_name)

In [191]:
pbar = tqdm(total=len(files), desc='File Name')
for file in files:
    file_name = os.path.basename(file)
    pbar.set_description(file_name)
    stats_df = pd.read_csv(file)
    stats_df = stats_df.dropna()
    stats_df = stats_df[(stats_df['MP'] >= 820) 
                        & ((stats_df.duplicated('Season', keep='last') & (stats_df['Tm']=='TOT'))) 
                        | (~stats_df.duplicated('Season'))
                       ].set_index('Season')
    
    data_split(stats_df, pre_mask, 'pre', file_name)
    data_split(stats_df, post_mask, 'post', file_name)
    
    pbar.update(1)

ömer_aşık.csv: 100%|██████████████████████████████████████████████████████████████▉| 1666/1667 [00:23<00:00, 96.71it/s]